In [ ]:
import pandas as pd
import numpy as np


# Assuming we have the task2_stocks.csv file already available in the environment
file_path = 'task2_stocks.csv'  # Update with actual path if needed

# Load the CSV data
df = pd.read_csv(file_path)

# Filter for AAPL stock only
df_aapl = df[df['Ticker'] == 'AAPL'].copy()

# Sort data by date for accurate calculation of indicators
df_aapl['Date'] = pd.to_datetime(df_aapl['Date'])
df_aapl = df_aapl.sort_values('Date').reset_index(drop=True)



# Define RSI calculation
def calculate_rsi(df, column='Adj Close', period=14):
    delta = df[column].diff()
    gain = np.where(delta > 0, delta, 0)
    loss = np.where(delta < 0, -delta, 0)

    avg_gain = pd.Series(gain).rolling(window=period).mean()
    avg_loss = pd.Series(loss).rolling(window=period).mean()

    rs = avg_gain / avg_loss
    rsi = 100 - (100 / (1 + rs))

    return rsi

# Define MACD calculation
def calculate_macd(df, column='Adj Close', short_period=12, long_period=26, signal_period=9):
    short_ema = df[column].ewm(span=short_period, adjust=False).mean()
    long_ema = df[column].ewm(span=long_period, adjust=False).mean()
    macd = short_ema - long_ema
    macd_signal = macd.ewm(span=signal_period, adjust=False).mean()
    macd_hist = macd - macd_signal

    return macd, macd_signal, macd_hist

# Define Momentum calculation
def calculate_momentum(df, column='Adj Close', period=10):
    momentum = df[column].diff(period)
    return momentum

# Define Ultimate Oscillator calculation
def calculate_ultimate_oscillator(df, period1=7, period2=14, period3=28):
    high_low = df['High'] - df['Low']
    high_close_prev = abs(df['High'] - df['Close'].shift(1))
    low_close_prev = abs(df['Low'] - df['Close'].shift(1))

    true_range = high_low.combine(high_close_prev, max).combine(low_close_prev, max)

    bp = df['Close'] - np.minimum(df['Low'], df['Close'].shift(1))
    avg7 = bp.rolling(window=period1).sum() / true_range.rolling(window=period1).sum()
    avg14 = bp.rolling(window=period2).sum() / true_range.rolling(window=period2).sum()
    avg28 = bp.rolling(window=period3).sum() / true_range.rolling(window=period3).sum()

    ultimate_oscillator = 100 * ((4 * avg7) + (2 * avg14) + avg28) / (4 + 2 + 1)

    return ultimate_oscillator

# Apply indicators to AAPL DataFrame
df_aapl['RSI'] = calculate_rsi(df_aapl)
df_aapl['MACD'], df_aapl['MACD_Signal'], df_aapl['MACD_Hist'] = calculate_macd(df_aapl)
df_aapl['Momentum'] = calculate_momentum(df_aapl)
df_aapl['Ultimate Oscillator'] = calculate_ultimate_oscillator(df_aapl)


In [ ]:
# Filtering out rows with NaN values and dropping 'Open', 'High', and 'Low' columns
df_aapl_filtered = df_aapl.dropna().drop(columns=['Open', 'High', 'Low'])

# Creating the 'buy' column
# If the adjusted close price in 5 days is higher than today's adjusted close, set 'buy' to 1, else 0
df_aapl_filtered['buy'] = (df_aapl_filtered['Adj Close'].shift(-5) > df_aapl_filtered['Adj Close']).astype(int)
df_aapl_filtered

,Date,Close,Adj Close,Volume,Ticker,RSI,MACD,MACD_Signal,MACD_Hist,Momentum,Ultimate Oscillator,buy
28,2020-02-12,81.800003,79.579346,113730400,AAPL,55.958395,1.033660,0.954355,0.079304,0.882362,66.317622,0
29,2020-02-13,81.217499,79.012642,94747600,AAPL,54.899557,1.059350,0.975354,0.083996,0.429703,62.107362,0
30,2020-02-14,81.237503,79.032112,80113600,AAPL,62.796070,1.068958,0.994075,0.074883,3.933449,64.843759,0
31,2020-02-18,79.750000,77.585007,152531200,AAPL,51.646919,0.948866,0.985033,-0.036168,2.692589,58.196355,0
32,2020-02-19,80.904999,78.708641,93984000,AAPL,50.039552,0.933597,0.974746,-0.041149,1.343727,63.006012,0
...,...,...,...,...,...,...,...,...,...,...,...,...
992,2023-12-11,193.179993,192.449707,60943700,AAPL,54.474897,3.444939,3.484142,-0.039203,3.377167,59.080977,0
993,2023-12-12,194.710007,193.973953,52696900,AAPL,60.149668,3.422158,3.471745,-0.049588,4.293732,61.489651,0
994,2023-12-13,197.960007,197.211655,70404200,AAPL,64.692911,3.623588,3.502114,0.121474,8.557526,66.821153,0
995,2023-12-14,198.110001,197.361069,66831600,AAPL,68.983202,3.752028,3.552097,0.199931,8.129135,64.310054,0


In [28]:
import tensorflow as tf
from sklearn.preprocessing import StandardScaler
import numpy as np

# Remove the last 5 rows and reset index
df_aapl_filtered = df_aapl_filtered.iloc[:-5].reset_index(drop=True)

# Define features and target
features = ['Close', 'RSI', 'MACD', 'MACD_Signal', 'MACD_Hist', 'Momentum', 'Ultimate Oscillator']
X = df_aapl_filtered[features]
y = df_aapl_filtered['buy']

# Normalize features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Set up the time series sequences
sequence_length = 10  # Use past 10 days
X_seq, y_seq = [], []

for i in range(len(X_scaled) - sequence_length):
    X_seq.append(X_scaled[i:i + sequence_length])  # past 10 days data as features
    y_seq.append(y.iloc[i + sequence_length])      # target is the 'buy' value on the 11th day

X_seq, y_seq = np.array(X_seq), np.array(y_seq)

# Define test start and end indices for the expanding window
test_start = 850
test_end = 900

# Store results
test_acc_list = []

# Loop for each test point from 801 to 900
for i in range(test_start, test_end + 1):
    # Define expanding training set up to the current index
    X_train = X_seq[:i]
    y_train = y_seq[:i]
    X_test = X_seq[i:i + 1]  # Single day
    y_test = y_seq[i:i + 1]

    # Define the CNN model
    model = tf.keras.models.Sequential([
        tf.keras.layers.Conv1D(filters=64, kernel_size=2, activation='relu', input_shape=(sequence_length, X_train.shape[2])),
        tf.keras.layers.MaxPooling1D(pool_size=2),
        tf.keras.layers.Conv1D(filters=32, kernel_size=2, activation='relu'),
        tf.keras.layers.MaxPooling1D(pool_size=2),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(50, activation='relu'),
        tf.keras.layers.Dense(1, activation='sigmoid')  # Sigmoid for binary classification
    ])

    # Compile the model
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

    # Train the model for 30 epochs
    print(f"\nTraining with training set up to index {i - 1} and evaluating on index {i}")
    model.fit(X_train, y_train, epochs=5, batch_size=16, verbose=1)

    # Evaluate the model on the single test day
    test_loss, test_acc = model.evaluate(X_test, y_test, verbose=0)
    test_acc_list.append(test_acc)
    print(f"Day {i}: Test Accuracy = {test_acc}")

# Calculate and print the average test accuracy across all days
average_test_acc = np.mean(test_acc_list)
print("\nAverage Test Accuracy from Day 801 to 900:", average_test_acc)


/usr/local/lib/python3.10/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)



Training with training set up to index 849 and evaluating on index 850
Epoch 1/5
54/54 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.5641 - loss: 0.6813
Epoch 2/5
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.5986 - loss: 0.6633
Epoch 3/5
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.6619 - loss: 0.6296
Epoch 4/5
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.6716 - loss: 0.6198
Epoch 5/5
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.6703 - loss: 0.6126
Day 850: Test Accuracy = 1.0

Training with training set up to index 850 and evaluating on index 851
Epoch 1/5
54/54 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.6149 - loss: 0.6573
Epoch 2/5
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.6203 - loss: 0.6476
Epoch 3/5
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.6693 - loss: 0.6227
Epoch 4/5
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.6743 - loss: 0.6102
Epoch 5/5
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.6897 - loss: 0.59

In [29]:
import tensorflow as tf
from sklearn.preprocessing import StandardScaler
import numpy as np
import pandas as pd

# Remove the last 5 rows and reset index
df_aapl_filtered = df_aapl_filtered.iloc[:-5].reset_index(drop=True)

# Define features and target
features = ['Close', 'RSI', 'MACD', 'MACD_Signal', 'MACD_Hist', 'Momentum', 'Ultimate Oscillator']
X = df_aapl_filtered[features]
y = df_aapl_filtered['buy']

# Normalize features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Set up the time series sequences
sequence_length = 10  # Use past 10 days
X_seq, y_seq = [], []

for i in range(len(X_scaled) - sequence_length):
    X_seq.append(X_scaled[i:i + sequence_length])  # past 10 days data as features
    y_seq.append(y.iloc[i + sequence_length])      # target is the 'buy' value on the 11th day

X_seq, y_seq = np.array(X_seq), np.array(y_seq)

# Define test start and end indices for 5-day blocks
test_start = 801
test_end = 930

# Initialize predictions column in the original DataFrame
df_aapl_filtered['prediction'] = np.nan

# Loop through each 5-day prediction window
for i in range(test_start, test_end + 1, 5):
    # Define expanding training set up to the current index
    X_train = X_seq[:i]
    y_train = y_seq[:i]
    X_test = X_seq[i:i + 5]  # Next 5 days for prediction

    # Define the CNN model
    model = tf.keras.models.Sequential([
        tf.keras.layers.Conv1D(filters=64, kernel_size=2, activation='relu', input_shape=(sequence_length, X_train.shape[2])),
        tf.keras.layers.MaxPooling1D(pool_size=2),
        tf.keras.layers.Conv1D(filters=32, kernel_size=2, activation='relu'),
        tf.keras.layers.MaxPooling1D(pool_size=2),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(50, activation='relu'),
        tf.keras.layers.Dense(1, activation='sigmoid')  # Sigmoid for binary classification
    ])

    # Compile and train the model
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    model.fit(X_train, y_train, epochs=5, batch_size=16, verbose=1)

    # Generate predictions for the next 5 days
    predictions = model.predict(X_test).flatten()

    # Store the predictions back into the original DataFrame
    df_aapl_filtered.loc[i + sequence_length:i + sequence_length + 4, 'prediction'] = predictions

# Display the DataFrame with predictions
print(df_aapl_filtered[['Date', 'buy', 'prediction']].tail(20))


Epoch 1/5


/usr/local/lib/python3.10/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


51/51 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.5929 - loss: 0.6740
Epoch 2/5
51/51 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.6118 - loss: 0.6505
Epoch 3/5
51/51 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.6284 - loss: 0.6338
Epoch 4/5
51/51 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.6507 - loss: 0.6249
Epoch 5/5
51/51 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.6802 - loss: 0.6025
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 116ms/step
Epoch 1/5


/usr/local/lib/python3.10/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


51/51 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.5818 - loss: 0.6707
Epoch 2/5
51/51 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.6404 - loss: 0.6427
Epoch 3/5
51/51 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.6239 - loss: 0.6338
Epoch 4/5
51/51 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.6810 - loss: 0.6186
Epoch 5/5
51/51 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.6610 - loss: 0.5999
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 97ms/step
Epoch 1/5


...

58/58 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.5392 - loss: 0.6817
Epoch 2/5
58/58 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.6314 - loss: 0.6468
Epoch 3/5
58/58 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.6842 - loss: 0.6168
Epoch 4/5
58/58 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.6858 - loss: 0.6213
Epoch 5/5
58/58 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.6738 - loss: 0.6059
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 109ms/step
Epoch 1/5


/usr/local/lib/python3.10/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


58/58 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.5418 - loss: 0.6904
Epoch 2/5
58/58 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.6490 - loss: 0.6485
Epoch 3/5
58/58 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.6436 - loss: 0.6352
Epoch 4/5
58/58 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.6675 - loss: 0.6091
Epoch 5/5
58/58 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7119 - loss: 0.5799


/usr/lib/python3.10/contextlib.py:153: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(typ, value, traceback)


ValueError: math domain error

In [31]:
df_aapl_filtered.tail(50)

,Date,Close,Adj Close,Volume,Ticker,RSI,MACD,MACD_Signal,MACD_Hist,Momentum,Ultimate Oscillator,buy,prediction
879,2023-08-10,177.970001,176.825150,54686900,AAPL,24.553693,-2.230426,0.032167,-2.262594,-15.151932,26.081248,0,0.313292
880,2023-08-11,177.789993,176.884857,51988100,AAPL,22.434627,-2.638519,-0.501970,-2.136549,-17.685425,28.332647,0,0.251857
881,2023-08-14,179.460007,178.546371,43675600,AAPL,24.697356,-2.795640,-0.960704,-1.834936,-16.639908,32.224822,0,0.163717
882,2023-08-15,177.449997,176.546585,43622600,AAPL,20.647179,-3.046407,-1.377845,-1.668563,-17.805099,37.441234,0,0.180028
883,2023-08-16,176.570007,175.671066,46964900,AAPL,20.938797,-3.278003,-1.757876,-1.520127,-15.670120,37.793791,1,0.187729
884,2023-08-17,174.000000,173.114151,66062900,AAPL,11.710541,-3.626066,-2.131514,-1.494552,-16.826096,29.768756,1,0.202567
885,2023-08-18,174.490005,173.601669,61114200,AAPL,11.303821,-3.818553,-2.468922,-1.349631,-7.217651,35.097160,1,0.193149
886,2023-08-21,175.839996,174.944794,46311900,AAPL,15.830985,-3.818701,-2.738878,-1.079823,-2.754730,42.351156,1,0.384552
887,2023-08-22,177.229996,176.327698,42084200,AAPL,21.959189,-3.664983,-2.924099,-0.740884,-2.315704,44.018561,1,0.366074
888,2023-08-23,181.119995,180.197922,52722800,AAPL,33.340796,-3.194046,-2.978088,-0.215957,3.154144,47.949240,1,0.365870


In [32]:
df_aapl_filtered.to_csv('AAPL_predictions.csv', index=False)